# Setup Code

In [1]:
# Importing all packages used
import pandas as pd

## Importing delivery_radius_log

In [5]:
# Importing the CSV file delivery_radius_log as a dataframe first
df_delivery_radius_log = pd.read_csv('data/delivery_radius_log.csv')
df_delivery_radius_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316 entries, 0 to 1315
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DELIVERY_AREA_ID         1316 non-null   object
 1   DELIVERY_RADIUS_METERS   1316 non-null   int64 
 2   EVENT_STARTED_TIMESTAMP  1316 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.0+ KB


In [6]:
df_delivery_radius_log.head()

,DELIVERY_AREA_ID,DELIVERY_RADIUS_METERS,EVENT_STARTED_TIMESTAMP
0,5db02e5d401d690c836b9ead,3000,2022-06-14T08:26:20.923854Z
1,5db02e5d401d690c836b9ead,7000,2022-06-14T08:49:01.186365Z
2,5db02e5d401d690c836b9ead,3000,2022-06-18T07:43:57.662294Z
3,5db02e5d401d690c836b9ead,7000,2022-06-18T08:00:45.227506Z
4,5d78a7e552dfabd5251dab7b,4000,2022-06-18T08:05:29.093983Z


## Importing purchases 

In [8]:
# Importing the CSV file delivery_radius_log as a dataframe first
df_purchases = pd.read_csv('data/purchases.csv')
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177895 entries, 0 to 177894
Data columns (total 6 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   PURCHASE_ID                            177895 non-null  object 
 1   TIME_RECEIVED                          177895 non-null  object 
 2   TIME_DELIVERED                         177895 non-null  object 
 3   END_AMOUNT_WITH_VAT_EUR                177895 non-null  float64
 4   DROPOFF_DISTANCE_STRAIGHT_LINE_METRES  177895 non-null  int64  
 5   DELIVERY_AREA_ID                       177895 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 8.1+ MB


In [9]:
df_purchases.head()

,PURCHASE_ID,TIME_RECEIVED,TIME_DELIVERED,END_AMOUNT_WITH_VAT_EUR,DROPOFF_DISTANCE_STRAIGHT_LINE_METRES,DELIVERY_AREA_ID
0,5f85beff7762a1539ad6faf1,2022-10-13T14:51:43.048Z,2022-10-13T15:18:35.265Z,17.87,735,5d78a7e552dfabd5251dab7b
1,5f85c08dddf0c9826389f3cd,2022-10-13T14:58:21.078Z,2022-10-13T15:28:09.194Z,17.75,436,5cc1b60b034adf90cd8f14dd
2,5f85bc2cf49ddea98955ce5f,2022-10-13T14:39:40.153Z,2022-10-13T15:05:15.058Z,25.80,867,5cc1b60b034adf90cd8f14dd
3,5f855dbf5a93deaf2be5b872,2022-10-13T07:56:47.003Z,2022-10-13T09:05:14.37Z,15.70,252,5db02e5d401d690c836b9ead
4,5f85be8a8876393ee141ed82,2022-10-13T14:49:46.693Z,2022-10-13T15:14:31.299Z,18.80,857,5db02e5d401d690c836b9ead


# Q1